In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import pandas as pd
from nltk.corpus import stopwords
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup

In [2]:
#the file shows the fields separated by space and quoted by ", 
#and that where someone has included a quoted string in their review 
#there is a backslash before the quotes to escape them
df = pd.read_csv('data.tsv', sep=' ', quotechar='"', escapechar='\\')
print(df.shape)
df.head(3)

(50000, 3)


,new_id,sentiment,review
0,1,1,I quite enjoyed this movie for two reasons. Th...
1,2,0,A man in blackface lands in a spaceship and me...
2,3,1,I was really beginning to enjoy this show. It ...


In [3]:
# Split the data into training and testing set based on the splits,csv file
# Each column consist of 25,000 test samples ID, so we can separate the data into three different train/test samples
split = pd.read_csv('splits.csv', sep='\t', quotechar='"', escapechar='\\')
split.head(3)

,split_1,split_2,split_3
0,2,9245,8403
1,3,35118,40376
2,4,28666,19247


In [4]:
split.shape

(25000, 3)

### Train and test split

In [5]:
# Use left join to seperate our data 
test1 = df.merge(split[['split_1']], left_on='new_id', right_on='split_1', how='inner')
test1 = test1.drop('split_1',axis = 1)
test1 = test1.reset_index(drop=True)

#Select the rows not in test1 id
train1 = df.loc[~df['new_id'].isin(test1['new_id'])]
train1 = train1.reset_index(drop=True)

# Use left join to seperate our data 
test2 = df.merge(split[['split_2']], left_on='new_id', right_on='split_2', how='inner')
test2 = test2.drop('split_2',axis = 1)
test2 = test2.reset_index(drop=True)

#Select the rows not in test1 id
train2 = df.loc[~df['new_id'].isin(test2['new_id'])]
train2 = train2.reset_index(drop=True)

# Use left join to seperate our data 
test3 = df.merge(split[['split_3']], left_on='new_id', right_on='split_3', how='inner')
test3 = test3.drop('split_3',axis = 1)
test3 = test3.reset_index(drop=True)

#Select the rows not in test1 id
train3 = df.loc[~df['new_id'].isin(test3['new_id'])]
train3 = train3.reset_index(drop=True)


In [6]:
train1 = train1.reset_index(drop=True)
train1.head()

,new_id,sentiment,review
0,1,1,I quite enjoyed this movie for two reasons. Th...
1,5,0,I am a big fan of Stephen King. I loved The Ru...
2,6,0,"This is a terrible film, and not one scene has..."
3,7,0,"This weekend just passed I watched ""28 Weeks L..."
4,9,1,Gundam Wing is an amazing show from start to f...


In [7]:
example1 = BeautifulSoup(train1["review"][0])  
example1.get_text()

"I quite enjoyed this movie for two reasons. The first is that it gives an insight into the world of loyalism in northern ireland, which is very rarely treated in movies, most of which tell us about the republican struggle. The second reason is the performances of the actors. I thought they gave very honest and convincing portrayals of a very seedy underworld that not many people hear about outside my native shores.All in all, it is an entertaining ganster movie with stellar performances from a who's who in northern irish actors cast. It wont move the earth, although it may slightly open some peoples eyes to the murky world of loyalist paramilitaries."

In [8]:
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search

In [9]:
letters_only

'I quite enjoyed this movie for two reasons  The first is that it gives an insight into the world of loyalism in northern ireland  which is very rarely treated in movies  most of which tell us about the republican struggle  The second reason is the performances of the actors  I thought they gave very honest and convincing portrayals of a very seedy underworld that not many people hear about outside my native shores All in all  it is an entertaining ganster movie with stellar performances from a who s who in northern irish actors cast  It wont move the earth  although it may slightly open some peoples eyes to the murky world of loyalist paramilitaries '

In [10]:
lower_case = letters_only.lower()
words = lower_case.split()  

In [11]:
import nltk
# nltk.download()  # Download text data sets, including stop words

In [12]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [14]:
clean_review = review_to_words( train1["review"][0] )
print(clean_review)

quite enjoyed movie two reasons first gives insight world loyalism northern ireland rarely treated movies tell us republican struggle second reason performances actors thought gave honest convincing portrayals seedy underworld many people hear outside native shores entertaining ganster movie stellar performances northern irish actors cast wont move earth although may slightly open peoples eyes murky world loyalist paramilitaries


In [15]:
# Get the number of reviews based on the dataframe column size
num_reviews = train1["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range(num_reviews):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train1["review"][i] ) )


In [16]:
len(clean_train_reviews)

25000

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()


In [18]:
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
print(train_data_features.shape)

(25000, 5000)


In [23]:
vocab = vectorizer.vocabulary_
len(vocab)

5000

In [21]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
# for tag, count in zip(vocab, dist):
#     print(count, tag)

In [22]:
train_data_features.shape

(25000, 5000)

In [23]:
# from sklearn.ensemble import RandomForestClassifier
# # Initialize a Random Forest classifier with 100 trees
# forest = RandomForestClassifier(n_estimators = 100) 
# # Fit the forest to the training set, using the bag of words as 
# # features and the sentiment labels as the response variable
# #
# # This may take a few minutes to run
# forest = forest.fit( train_data_features, train1["sentiment"] )

In [24]:
# ### Save the model I train in pickle 
# # Save the final model in trained_model.pkl
# import pickle

# # data2 = grid_sgdlogreg2
# output = open('trained_model.pkl', 'wb')

# # Pickle dictionary using protocol 0.
# pickle.dump(forest, output)

# output.close()

In [25]:
# Read the model from trained_model.pkl


In [26]:
test1.head()

,new_id,sentiment,review
0,2,0,A man in blackface lands in a spaceship and me...
1,3,1,I was really beginning to enjoy this show. It ...
2,4,0,It is one of the joys of Shakespeare that ther...
3,8,0,I don't think it's necessary to outline the pl...
4,13,0,I read all of the other comments which made th...


In [27]:
# # Create an empty list and append the clean reviews one by one
# num_reviews = len(test1["review"])
# clean_test_reviews = [] 


# for i in range(num_reviews):
# #     if( (i+1) % 1000 == 0 ):
# #         print "Review %d of %d\n" % (i+1, num_reviews)
#     clean_review = review_to_words( test1["review"][i] )
#     clean_test_reviews.append( clean_review )

# # Get a bag of words for the test set, and convert to a numpy array
# test_data_features = vectorizer.transform(clean_test_reviews)
# test_data_features = test_data_features.toarray()

# # Use the random forest to make sentiment label predictions
# result = forest.predict(test_data_features)

# # # Copy the results to a pandas dataframe with an "id" column and
# # # a "sentiment" column
# # output = pd.DataFrame( data={"id":test1["id"], "sentiment":result} )

# # # Use pandas to write the comma-separated output file
# # output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [28]:

# from sklearn.metrics import roc_auc_score
# y_true = np.array(test1['sentiment'])
# y_scores = result
# roc_auc_score(y_true, y_scores)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

train_data_features = tfv.fit_transform(clean_train_reviews)
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

# Create an empty list and append the clean reviews one by one
num_reviews = len(test1["review"])
clean_test_reviews = [] 

for i in range(num_reviews):
#     if( (i+1) % 1000 == 0 ):
#         print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test1["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = tfv.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

model = LogisticRegression(penalty='l2', dual=True, tol=0.0001, 
                         C=1, fit_intercept=True, intercept_scaling=1.0, 
                         class_weight=None, random_state=None)



In [ ]:
model.fit(train_data_features, train1["sentiment"])
pre_test_y = model.predict(test_data_features)

from sklearn.metrics import roc_auc_score
y_true = np.array(test1['sentiment'])
y_scores = result
roc_auc_score(y_true, y_scores)